In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv('df.tsv', sep='\t')

data['DrugName_encoded'] = data['Drug Name'].astype('category').cat.codes

data['Targets_split'] = data['Targets'].fillna('').apply(lambda x: x.split(','))

all_targets = set([target for sublist in data['Targets_split'] for target in sublist])
target_to_id = {target: idx for idx, target in enumerate(all_targets)}

data['Targets_encoded'] = data['Targets_split'].apply(lambda x: [target_to_id[t] for t in x])

data.head()

,ID,Drug Name,Targets,Z Score,Count,DrugName_encoded,Targets_split,Targets_encoded
0,1821,765771,NaN,-2.608364,279,13,[],[0]
1,1008,Methotrexate,Antimetabolite,-1.636781,279,159,[Antimetabolite],[23]
2,1168,Erlotinib,EGFR,-1.559059,279,94,[EGFR],[220]
3,1828,720427,NaN,-1.529078,279,9,[],[0]
4,1915,AZD3759,EGFR,-1.384414,279,27,[EGFR],[220]


# Eğitim

In [ ]:
num_drugs = len(data['DrugName_encoded'].unique())
num_targets = len(target_to_id)
embedding_dim = 16

drug_input = Input(shape=(1,), name='Drug_Input')
target_input = Input(shape=(None,), name='Target_Input')

drug_embedding = Embedding(input_dim=num_drugs, output_dim=embedding_dim, name='Drug_Embedding')(drug_input)
target_embedding = Embedding(input_dim=num_targets, output_dim=embedding_dim, name='Target_Embedding')(target_input)

target_vec = GlobalAveragePooling1D()(target_embedding)
drug_vec = Flatten()(drug_embedding)

combined = Concatenate()([drug_vec, target_vec])

dense1 = Dense(64, activation='relu')(combined)
dense2 = Dense(32, activation='relu')(dense1)
output = Dense(1, activation='linear', name='ZScore_Prediction')(dense2)

model = Model(inputs=[drug_input, target_input], outputs=output)

learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [ ]:
X_drug = data['DrugName_encoded'].values
X_target = pad_sequences(data['Targets_encoded'], padding='post')  # Target ID'leri padding ile düzenle
y = data['Z Score']

X_drug_train, X_drug_test, X_target_train, X_target_test, y_train, y_test = train_test_split(
    X_drug, X_target, y, test_size=0.2, random_state=42
)

In [ ]:
history = model.fit(
    x=[X_drug_train, X_target_train],
    y=y_train,
    batch_size=8,
    epochs=50,
    validation_data=([X_drug_test, X_target_test], y_test)
)

loss, mae = model.evaluate([X_drug_test, X_target_test], y_test)
print(f"Test Loss: {loss}")
print(f"Test MAE: {mae}")

Epoch 1/50
28/28 [==============================] - 1s 8ms/step - loss: 0.3826 - mae: 0.4783 - val_loss: 0.3681 - val_mae: 0.4994
Epoch 2/50
28/28 [==============================] - 0s 3ms/step - loss: 0.2121 - mae: 0.3589 - val_loss: 0.3651 - val_mae: 0.4892
Epoch 3/50
28/28 [==============================] - 0s 3ms/step - loss: 0.1417 - mae: 0.2916 - val_loss: 0.4093 - val_mae: 0.4989
Epoch 4/50
28/28 [==============================] - 0s 3ms/step - loss: 0.0657 - mae: 0.1806 - val_loss: 0.3487 - val_mae: 0.4815
Epoch 5/50
28/28 [==============================] - 0s 3ms/step - loss: 0.0402 - mae: 0.1335 - val_loss: 0.3560 - val_mae: 0.4869
Epoch 6/50
28/28 [==============================] - 0s 3ms/step - loss: 0.0264 - mae: 0.0992 - val_loss: 0.3577 - val_mae: 0.4865
Epoch 7/50
28/28 [==============================] - 0s 3ms/step - loss: 0.0166 - mae: 0.0768 - val_loss: 0.3588 - val_mae: 0.4852
Epoch 8/50
28/28 [==============================] - 0s 3ms/step - loss: 0.0146 - mae: 0.06

In [ ]:
model.save("451Lu.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
loaded_model = load_model("22RV1.h5")

In [ ]:
new_data = {
    'Drug Name': 'MK-8776',  # Örnek ilaç
    'Targets': ['CHEK1', 'CHEK2', 'CDK2']  # Örnek hedefler
}


new_drug_encoded = data['Drug Name'].astype('category').cat.categories.get_loc(new_data['Drug Name'])


new_targets_encoded = [target_to_id.get(target, 0) for target in new_data['Targets']]
new_targets_padded = pad_sequences([new_targets_encoded], padding='post', maxlen=X_target.shape[1])


prediction = model.predict([np.array([new_drug_encoded]), new_targets_padded])

print(f"Tahmin edilen Z Skor: {prediction[0][0]}")


1/1 [==============================] - 0s 23ms/step
Tahmin edilen Z Skor: 0.5538578629493713


In [ ]:
import pandas as pd
import json
import os


dataset_dir = "datasets/"
drug_set = set()
target_set = set()

# Tüm veri setlerini işleme
for file in os.listdir(dataset_dir):
    if file.endswith(".tsv"):
        df = pd.read_csv(os.path.join(dataset_dir, file), sep='\t')

        drug_set.update(df['Drug Name'].unique())
        for targets in df['Targets']:

            if isinstance(targets, str) and targets:
                target_set.update(targets.split(','))ı

drug_to_id = {drug: idx for idx, drug in enumerate(sorted(drug_set))}
target_to_id = {target.strip(): idx for idx, target in enumerate(sorted(target_set))}

with open("cell_drug_encoder.json", "w") as f:
    json.dump(drug_to_id, f)

with open("cell_target_encoder.json", "w") as f:
    json.dump(target_to_id, f)

print("Encoder dosyaları oluşturuldu ve kaydedildi!")

Encoder dosyaları oluşturuldu ve kaydedildi!
